Carregando imagem e transformando em cinza


In [1]:
import cv2
import numpy as np
from PIL import Image

# Carregue a imagema partir do disco
img = cv2.imread("cafe75.jpg")
# Converte a imagem para tons de cinza
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#imgdisplay = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#display(Image.fromarray(imgdisplay))



LIMIARIZAÇÃO

In [ ]:

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   # Aplique a limiarização com o limiar de 64
ret, img_threshold = cv2.threshold(img_gray, 130, 255, cv2.THRESH_BINARY)

    #Testes para tirar fundo da segmentação
inverted_mask = cv2.bitwise_not(img_threshold)

    #cv2.imshow("Result", inverted_mask)
    # Aplicar máscara invertida à imagem original
result = cv2.bitwise_and(img, img, mask=inverted_mask)

    #   Adicionar um quarto canal (canal alpha) à imagem resultante
result_with_alpha = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)

    # Definir a transparência para os pixels de fundo (onde a máscara binária é igual a 0)
result_with_alpha[:, :, 3] = inverted_mask

    # Salvar a imagem resultante com canal alpha em um arquivo PNG
cv2.imwrite('imagem_com_alphaaa.png', result_with_alpha)

    # Mostrar a imagem resultante com canal alpha
cv2.imshow('Result', result_with_alpha)
cv2.waitKey(0)
cv2.destroyAllWindows()




DETECÇÃO DE BORDAS


In [2]:
# Converte a imagem para escala de cinza
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplique o algoritmo Canny para detectar bordas
edges = cv2.Canny(gray, 50, 150)

# Mostre a imagem original e a imagem com bordas detectadas
cv2.imshow("Bordas", edges)
cv2.imwrite("Bordas.jpg", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


MÉTODO PIRAMIDAL

In [ ]:
# Aplique a transformada piramidal
levels = 4
pyramids = [img_gray]
for i in range(levels):
    pyramids.append(cv2.pyrDown(pyramids[i]))

# Aplique o algoritmo de segmentação desejado (por exemplo, o's binarization)
ret, thresh = cv2.threshold(pyramids[levels-1], 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


# Expandir a segmentação para a imagem original
for i in range(levels-1, 0, -1):
    thresh = cv2.pyrUp(thresh)
    thresh = cv2.resize(thresh, (pyramids[i].shape[1], pyramids[i].shape[0]))
    #thresh = cv2.bitwise_and(pyramids[i], thresh)

if thresh.shape != img.shape:
    thresh = cv2.resize(thresh, (img.shape[1], img.shape[0]))

if img.dtype != thresh.dtype:
    img2 = cv2.convertTo(thresh, img.dtype)



thresh = cv2.bitwise_and(img, img, mask=thresh)

# Exibir a imagem segmentada
cv2.imwrite("Piramidal-25-l4.jpg", thresh)
cv2.imshow("Segmentacao", thresh)
cv2.waitKey(0)

SEGMENTAÇÃO NO ESPECTRO HSV

In [6]:
# Converte a imagem para o espectro HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Defina os limites de tonalidade de cor (Hue) que deseja segmentar
lower = np.array([10, 30, 30])
upper = np.array([20, 255, 255])
# Crie uma máscara utilizando os limites de tonalidade de cor
mask = cv2.inRange(img_hsv, lower, upper)
# Aplique a máscara na imagem original
img_segmentada = cv2.bitwise_and(img, img, mask=mask)
# Exiba o resultado da segmentação
cv2.imshow("Segmentacao", img_segmentada)
cv2.imwrite("Método HSV graos75.png", img_segmentada)
cv2.waitKey(0)


-1

In [ ]:
# Converte a imagem para o espectro HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Defina os limites de tonalidade de cor (Hue) que deseja segmentar
lower = np.array([0, 0, 10])
upper = np.array([255, 255, 60])
# Crie uma máscara utilizando os limites de tonalidade de cor
mask = cv2.inRange(img_hsv, lower, upper)
# Aplique a máscara na imagem original
img_segmentada = cv2.bitwise_and(img, img, mask=mask)
# Exiba o resultado da segmentação
cv2.imshow("Segmentacao", img_segmentada)
cv2.imwrite("Método HSV graos.png", img_segmentada)
cv2.waitKey(0)

HSV COM A COR DO FUNDO


In [ ]:
# Converte a imagem para o espectro HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# Defina os limites de tonalidade de cor (Hue) que deseja segmentar
lower = np.array([0, 0, 100])
upper = np.array([180, 80, 255])
# Crie uma máscara utilizando os limites de tonalidade de cor
mask = cv2.inRange(img_hsv, lower, upper)
mask = cv2.bitwise_not(mask)
# Aplique a máscara na imagem original

img_segmentada = cv2.bitwise_and(img, img, mask=mask)
# Exiba o resultado da segmentação
cv2.imshow("Segmentacao", img_segmentada)
cv2.imwrite("Método HSV 25.png", img_segmentada)
cv2.waitKey(0)

K-MEANS

In [7]:
# A imagem é transformada em uma matriz unidimensional de dimensão 3, onde cada linha representa um pixel e cada coluna representa um canal de cor (vermelho, verde e azul).
Z = img.reshape((-1,3))
# converte para np.float32
Z = np.float32(Z)
# Defina as condições de parada do algoritmo K-Means
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# Defina o número de grupos em que deseja dividir os pixels da imagem
K = 2
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_PP_CENTERS)
# converte de volta para uint8
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

# Criar uma máscara com as áreas segmentadas em branco e o resto em preto
mask = np.zeros((img.shape[0]*img.shape[1],1), np.uint8)
majority_label = np.argmax(np.bincount(label.flatten()))
mask[label.flatten() == majority_label] = 255
mask = mask.reshape((img.shape[0],img.shape[1]))

# Aplicar a máscara na imagem original
inv_mask = cv2.bitwise_not(mask)
result = cv2.bitwise_and(img, img, mask=inv_mask)


    #   Adicionar um quarto canal (canal alpha) à imagem resultante
result_with_alpha = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)

    # Definir a transparência para os pixels de fundo (onde a máscara binária é igual a 0)
result_with_alpha[:, :, 3] = inv_mask


    # Salvar a imagem resultante com canal alpha em um arquivo PNG
cv2.imshow("Segmentacao", result_with_alpha)
cv2.imwrite('imagem_com_alphaaakmsssssssssssssssseans.png', result_with_alpha)

# Mostrar o resultado


cv2.waitKey(0)
cv2.destroyAllWindows()


DETECÇÃO DE BACIAS

In [12]:

ret, thresh = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# Remover os ruídos
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 3)
sure_bg = cv2.dilate(opening,kernel,iterations=15)
# Encontrar o marcador desconhecido e o marcador certo
dist_transform = cv2.distanceTransform(opening, cv2.DIST_C, 3)
ret, sure_fg = cv2.threshold(dist_transform,1*dist_transform.max(),255,0)
# encontrar marcador desconhecido
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
# Encontrar marcadores
ret, markers = cv2.connectedComponents(sure_fg)
# Adicione 1 a todos os marcadores para que os marcadores desconhecidos sejam 1, 2, 3, ...
markers = markers+1
# Marque o marcador desconhecido com 0
markers[unknown == 255] = 0
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
#cv2.imshow("Segmentacao", unknown)
#cv2.imshow("Segmentacao", img)
cv2.waitKey(0)

#Testes para tirar fundo da segmentação
# Inverter máscara de segmentação
#inverted_mask = cv2.bitwise_not(segmentation_mask)

# Aplicar máscara invertida à imagem original
result = cv2.bitwise_and(img, img, mask=unknown)


# Mostrar imagem resultante
cv2.imshow("Result", result)
cv2.imwrite("Deteccao de Bacias.jpg", result)
cv2.waitKey(0)
cv2.destroyAllWindows()


#Utilizando a função cv2.inpaint() para preencher as regiões do fundo da imagem com pixels interpolados.
#inverted_mask = cv2.bitwise_not(unknown)
#result2 = cv2.inpaint(img, inverted_mask, 3, cv2.INPAINT_TELEA)

# Mostrar imagem resultante
#cv2.imshow("Result", result2)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

True

Detecção de Pontos
ALGORITMO HARRIS

In [ ]:
# Aplique o detector de canto de Harris
gray = np.float32(img_gray)
dst = cv2.cornerHarris(img_gray, 4, 5, 0.02)

# Normalize o resultado para torná-lo mais visível
dst = cv2.dilate(dst, None)
img[dst > 0.01*dst.max()] = [0, 0, 255]

# Mostre a imagem com os pontos de canto
cv2.imshow('image_with_points', img)
cv2.imwrite("Deteccao de pontos harris.jpg", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

O algoritmo FAST

In [3]:
# Crie um objeto FAST
fast = cv2.FastFeatureDetector_create()

# Detecte os pontos
points = fast.detect(img_gray, None)

# Desenhe os pontos na imagem
image_with_points = cv2.drawKeypoints(img, points, outImage=None)

# Mostre a imagem com os pontos
cv2.imshow('image_with_points', image_with_points)
cv2.imwrite("Deteccao de pontos fast.jpg" , image_with_points)
cv2.waitKey(0)
cv2.destroyAllWindows()

Algoritmo ORB (Oriented FAST and Rotated BRIEF)

In [ ]:
# Crie um objeto ORB
orb = cv2.ORB_create()

# Detecte os pontos
points = orb.detect(img_gray, None)

# Calcule os descritores dos pontos
points, descriptors = orb.compute(img_gray, points)

# Desenhe os pontos na imagem
image_with_points = cv2.drawKeypoints(img, points, outImage=None)

# Mostre a imagem com os pontos
cv2.imshow('image_with_points', image_with_points)
cv2.imwrite("Deteccao de pontos orb.jpg" , image_with_points)
cv2.waitKey(0)
cv2.destroyAllWindows()



DETECÇÃO DE LINHAS


In [4]:

# Aplicar filtro de ruído gaussiano para suavizar a imagem
gray_line = cv2.GaussianBlur(img_gray, (5, 5), 0)

# Detectar as bordas da imagem usando o Canny Edge Detector
edges = cv2.Canny(gray_line, 50, 150)

# Executar a transformada de Hough para detectar as linhas
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=50)

# Desenhar as linhas detectadas na imagem de saída
output_image = img.copy()
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(output_image, (x1, y1), (x2, y2), (0, 0, 256), 2)

# Mostrar a imagem de saída
cv2.imshow('Linhas detectadas', output_image)
cv2.imwrite("Deteccao linhas.jpg" , output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Crescimento de regioes

In [ ]:

imgcolorida = cv2.imread("cafe75.jpg")

def region_growing(img, seed):

    output = np.zeros_like(img)

    queue = []
    queue.append(seed)

    #define a tolerância de cor
    tolerance = 70

    # laço até que a fila esteja vazia
    while len(queue) > 0:

        pixel = queue.pop(0)

        # checar se o pixel está dentro da imagem
        if (pixel[0] >= 0 and pixel[0] < img.shape[0] and
            pixel[1] >= 0 and pixel[1] < img.shape[1]):

            # Checar se o pixel já foi processado
            if output[pixel[0], pixel[1]] == 0:
                # Check if the pixel is within the tolerance range of the seed pixel
                if abs(int(img[pixel[0], pixel[1]]) - int(img[seed[0], seed[1]])) < tolerance:
                    # Add the pixel to the region and mark it as processed
                    output[pixel[0], pixel[1]] = 255

                    # Add the neighboring pixels to the queue
                    queue.append([pixel[0] - 1, pixel[1]])
                    queue.append([pixel[0] + 1, pixel[1]])
                    queue.append([pixel[0], pixel[1] - 1])
                    queue.append([pixel[0], pixel[1] + 1])

    #inverter saida
    output = 255 - output

    return output


# definir a função de callback do mouse
def mouse_callback(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Set the seed pixel location
        params['seed'] = (y, x)


img = cv2.imread('cafe75.jpg', 0)

# criar uma janela para exibir a imagem
cv2.namedWindow('Input')

params = {'seed': None}
cv2.setMouseCallback('Input', mouse_callback, params)


cv2.imshow('Input', img)
cv2.waitKey(0)

# pegar a semente atraves do mouse
seed = params['seed']

#chamar a função de segmentação
output = region_growing(img, seed)

# Mostrar imagem resultante
#cv2.imshow('Output', output)
inverted_mask = cv2.bitwise_not(output)
cv2.imshow("Result", inverted_mask)
result = cv2.bitwise_and(imgcolorida, imgcolorida, mask=inverted_mask)

# Mostrar imagem resultante
cv2.imshow("Result", result)
cv2.imwrite("Crescimento75.jpg", result)

cv2.waitKey(0)



In [6]:
import cv2
import numpy as np

def region_growing(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    output = np.zeros_like(img_gray)

    queue = []
    seed = (img_gray.shape[0] // 2, img_gray.shape[1] // 2)
    queue.append(seed)

    # define a tolerância de cor
    tolerance = 70

    # laço até que a fila esteja vazia
    while len(queue) > 0:
        pixel = queue.pop(0)

        # checar se o pixel está dentro da imagem
        if (pixel[0] >= 0 and pixel[0] < img_gray.shape[0] and
            pixel[1] >= 0 and pixel[1] < img_gray.shape[1]):
            # Checar se o pixel já foi processado
            if output[pixel[0], pixel[1]] == 0:
                # Check if the pixel is within the tolerance range of the seed pixel
                if abs(int(img_gray[pixel[0], pixel[1]]) - int(img_gray[seed[0], seed[1]])) < tolerance:
                    # Add the pixel to the region and mark it as processed
                    output[pixel[0], pixel[1]] = 255

                    # Add the neighboring pixels to the queue
                    queue.append([pixel[0] - 1, pixel[1]])
                    queue.append([pixel[0] + 1, pixel[1]])
                    queue.append([pixel[0], pixel[1] - 1])
                    queue.append([pixel[0], pixel[1] + 1])

    # inverter saida
    output = 255 - output

    inverted_mask = cv2.bitwise_not(output)
    result = cv2.bitwise_and(img, img, mask=inverted_mask)

    result_with_alpha = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result_with_alpha[:, :, 3] = inverted_mask
    cv2.imwrite('imagem_com_alphaaa.png', result_with_alpha)

    cv2.imshow("Result", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


img = cv2.imread("cafe75.jpg")
region_growing(img)
